### Week-4 practice solutions. SQL CookBook (Chapter 5-7)

##### Connect to MySQL database (Sakila)

In [1]:
config = {
  'user': 'root',
  'password': 'root',
  'host': '127.0.0.1',
  'port': 8889,
  'database': 'sakila',
  'raise_on_warnings': True
}

In [3]:
pip install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 2.2 MB/s eta 0:00:0000:0100:01m

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
# create a MySQL server connection object
import mysql.connector
mydb = mysql.connector.connect(**config)

In [5]:
# cursor object
my_cursor = mydb.cursor(dictionary=True)

##### Chapter-5

information_schema.tables->storing information about all the tables in that particular schema

information_schema.column->storing information about all the columns in that particular table

information_schema.key_column_usage-> storing information about all the keys and indexes in that particular schema

information_schema.table_constraints -> storing information about all the constraints in all the tables in that particular schema

In [6]:
my_cursor.execute(" SELECT table_name FROM information_schema.tables WHERE table_schema ='sakila' ")
my_result = my_cursor.fetchall()

In [7]:
for table in my_result:
    print(table)

{'table_name': 'actor'}
{'table_name': 'actor_info'}
{'table_name': 'address'}
{'table_name': 'category'}
{'table_name': 'char_data_types'}
{'table_name': 'city'}
{'table_name': 'country'}
{'table_name': 'customer'}
{'table_name': 'customer_list'}
{'table_name': 'date_time_types'}
{'table_name': 'film'}
{'table_name': 'film_actor'}
{'table_name': 'film_category'}
{'table_name': 'film_list'}
{'table_name': 'film_text'}
{'table_name': 'inventory'}
{'table_name': 'language'}
{'table_name': 'nicer_but_slower_film_list'}
{'table_name': 'number_data_types'}
{'table_name': 'payment'}
{'table_name': 'people'}
{'table_name': 'rental'}
{'table_name': 'sales_by_film_category'}
{'table_name': 'sales_by_store'}
{'table_name': 'staff'}
{'table_name': 'staff_list'}
{'table_name': 'store'}


In [8]:
# Just fetching the tables
print(f"Table: {my_result[0]['table_name']}")
print(f"Total number of tables: {len(my_result)}")

Table: actor
Total number of tables: 27


In [9]:
for table_num in range(len(my_result)):
    table_name = my_result[table_num]['table_name']
    my_cursor.execute(f" SELECT column_name, data_type, ordinal_position FROM information_schema.columns WHERE table_schema ='sakila' AND table_name='{table_name}' ")
    column_names = my_cursor.fetchall()
    print()
    print(f'column names for table: {table_name}')
    print('-------------------------------------')
    print(column_names)


column names for table: actor
-------------------------------------
[{'column_name': 'actor_id', 'data_type': 'smallint', 'ordinal_position': 1}, {'column_name': 'first_name', 'data_type': 'varchar', 'ordinal_position': 2}, {'column_name': 'last_name', 'data_type': 'varchar', 'ordinal_position': 3}, {'column_name': 'last_update', 'data_type': 'timestamp', 'ordinal_position': 4}]

column names for table: actor_info
-------------------------------------
[{'column_name': 'actor_id', 'data_type': 'smallint', 'ordinal_position': 1}, {'column_name': 'first_name', 'data_type': 'varchar', 'ordinal_position': 2}, {'column_name': 'last_name', 'data_type': 'varchar', 'ordinal_position': 3}, {'column_name': 'film_info', 'data_type': 'text', 'ordinal_position': 4}]

column names for table: address
-------------------------------------
[{'column_name': 'address_id', 'data_type': 'smallint', 'ordinal_position': 1}, {'column_name': 'address', 'data_type': 'varchar', 'ordinal_position': 2}, {'column_n

5.3 Listing Indexed Columns for a Table

In [10]:
my_cursor.execute("show index from actor;")
my_index= my_cursor.fetchall()
for idx in my_index:
    print(idx)


{'Table': 'actor', 'Non_unique': 0, 'Key_name': 'PRIMARY', 'Seq_in_index': 1, 'Column_name': 'actor_id', 'Collation': 'A', 'Cardinality': 200, 'Sub_part': None, 'Packed': None, 'Null': '', 'Index_type': 'BTREE', 'Comment': '', 'Index_comment': ''}
{'Table': 'actor', 'Non_unique': 1, 'Key_name': 'idx_actor_last_name', 'Seq_in_index': 1, 'Column_name': 'last_name', 'Collation': 'A', 'Cardinality': 121, 'Sub_part': None, 'Packed': None, 'Null': '', 'Index_type': 'BTREE', 'Comment': '', 'Index_comment': ''}


5.4 Listing Constraints on a Table

In [11]:
sql_query='''SELECT a.table_name,
       a.constraint_name,
       b.column_name,
       a.constraint_type
FROM information_schema.table_constraints AS a,
     information_schema.key_column_usage AS b
WHERE a.table_name = 'address'
AND a.table_schema = 'sakila'
and a.table_name = b.table_name
AND a.table_schema = b.table_schema
AND a.constraint_name = b.constraint_name;
'''

my_cursor.execute(sql_query)
constraints=my_cursor.fetchall()
for row in constraints:
    print(row)

{'table_name': 'address', 'constraint_name': 'PRIMARY', 'column_name': 'address_id', 'constraint_type': 'PRIMARY KEY'}
{'table_name': 'address', 'constraint_name': 'fk_address_city', 'column_name': 'city_id', 'constraint_type': 'FOREIGN KEY'}


5.5 Listing Foreign Keys Without Corresponding Indexes

In [12]:
sql_query='''SELECT * FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE 
WHERE TABLE_SCHEMA = 'sakila' AND TABLE_NAME = 'address';
'''

my_cursor.execute(sql_query)
foreign_keys=my_cursor.fetchall()
for row in foreign_keys:
    print(row)

{'CONSTRAINT_CATALOG': 'def', 'CONSTRAINT_SCHEMA': 'sakila', 'CONSTRAINT_NAME': 'PRIMARY', 'TABLE_CATALOG': 'def', 'TABLE_SCHEMA': 'sakila', 'TABLE_NAME': 'address', 'COLUMN_NAME': 'address_id', 'ORDINAL_POSITION': 1, 'POSITION_IN_UNIQUE_CONSTRAINT': None, 'REFERENCED_TABLE_SCHEMA': None, 'REFERENCED_TABLE_NAME': None, 'REFERENCED_COLUMN_NAME': None}
{'CONSTRAINT_CATALOG': 'def', 'CONSTRAINT_SCHEMA': 'sakila', 'CONSTRAINT_NAME': 'fk_address_city', 'TABLE_CATALOG': 'def', 'TABLE_SCHEMA': 'sakila', 'TABLE_NAME': 'address', 'COLUMN_NAME': 'city_id', 'ORDINAL_POSITION': 1, 'POSITION_IN_UNIQUE_CONSTRAINT': 1, 'REFERENCED_TABLE_SCHEMA': 'sakila', 'REFERENCED_TABLE_NAME': 'city', 'REFERENCED_COLUMN_NAME': 'city_id'}


In [13]:
sql_query='''SHOW INDEX FROM sakila.address;
'''

my_cursor.execute(sql_query)
foreign_index=my_cursor.fetchall()
for row in foreign_index:
    print(row)

{'Table': 'address', 'Non_unique': 0, 'Key_name': 'PRIMARY', 'Seq_in_index': 1, 'Column_name': 'address_id', 'Collation': 'A', 'Cardinality': 603, 'Sub_part': None, 'Packed': None, 'Null': '', 'Index_type': 'BTREE', 'Comment': '', 'Index_comment': ''}
{'Table': 'address', 'Non_unique': 1, 'Key_name': 'idx_fk_city_id', 'Seq_in_index': 1, 'Column_name': 'city_id', 'Collation': 'A', 'Cardinality': 599, 'Sub_part': None, 'Packed': None, 'Null': '', 'Index_type': 'BTREE', 'Comment': '', 'Index_comment': ''}
{'Table': 'address', 'Non_unique': 1, 'Key_name': 'idx_location', 'Seq_in_index': 1, 'Column_name': 'location', 'Collation': 'A', 'Cardinality': 603, 'Sub_part': 32, 'Packed': None, 'Null': '', 'Index_type': 'SPATIAL', 'Comment': '', 'Index_comment': ''}


Here, we can see that there is an index present on the foreign key on the column city_id. In MySQL, when you define a foreign key constraint, an index on the columns involved is automatically created if it does not exist. 


5.6 Using SQL to Generate SQL 

In [14]:
sql_query='''SELECT CONCAT('SELECT COUNT(*) FROM ', table_schema, '.', table_name, ';') 
FROM information_schema.tables 
WHERE table_schema = 'sakila';
'''

my_cursor.execute(sql_query)
cnt=my_cursor.fetchall()
for row in cnt:
    print(list(row.values())[0])

SELECT COUNT(*) FROM sakila.actor;
SELECT COUNT(*) FROM sakila.actor_info;
SELECT COUNT(*) FROM sakila.address;
SELECT COUNT(*) FROM sakila.category;
SELECT COUNT(*) FROM sakila.char_data_types;
SELECT COUNT(*) FROM sakila.city;
SELECT COUNT(*) FROM sakila.country;
SELECT COUNT(*) FROM sakila.customer;
SELECT COUNT(*) FROM sakila.customer_list;
SELECT COUNT(*) FROM sakila.date_time_types;
SELECT COUNT(*) FROM sakila.film;
SELECT COUNT(*) FROM sakila.film_actor;
SELECT COUNT(*) FROM sakila.film_category;
SELECT COUNT(*) FROM sakila.film_list;
SELECT COUNT(*) FROM sakila.film_text;
SELECT COUNT(*) FROM sakila.inventory;
SELECT COUNT(*) FROM sakila.language;
SELECT COUNT(*) FROM sakila.nicer_but_slower_film_list;
SELECT COUNT(*) FROM sakila.number_data_types;
SELECT COUNT(*) FROM sakila.payment;
SELECT COUNT(*) FROM sakila.people;
SELECT COUNT(*) FROM sakila.rental;
SELECT COUNT(*) FROM sakila.sales_by_film_category;
SELECT COUNT(*) FROM sakila.sales_by_store;
SELECT COUNT(*) FROM sakila.

In [15]:
sql_query='''SELECT CONCAT('ALTER TABLE ', table_name,' DROP FOREIGN KEY ', constraint_name, ';') 
FROM information_schema.KEY_COLUMN_USAGE 
WHERE table_schema = 'sakila' AND REFERENCED_TABLE_NAME IS NOT NULL;
'''

my_cursor.execute(sql_query)
cnt=my_cursor.fetchall()
for row in cnt:
    print(list(row.values())[0])

ALTER TABLE address DROP FOREIGN KEY fk_address_city;
ALTER TABLE city DROP FOREIGN KEY fk_city_country;
ALTER TABLE customer DROP FOREIGN KEY fk_customer_address;
ALTER TABLE customer DROP FOREIGN KEY fk_customer_store;
ALTER TABLE film DROP FOREIGN KEY fk_film_language;
ALTER TABLE film DROP FOREIGN KEY fk_film_language_original;
ALTER TABLE film_actor DROP FOREIGN KEY fk_film_actor_actor;
ALTER TABLE film_actor DROP FOREIGN KEY fk_film_actor_film;
ALTER TABLE film_category DROP FOREIGN KEY fk_film_category_category;
ALTER TABLE film_category DROP FOREIGN KEY fk_film_category_film;
ALTER TABLE inventory DROP FOREIGN KEY fk_inventory_film;
ALTER TABLE inventory DROP FOREIGN KEY fk_inventory_store;
ALTER TABLE payment DROP FOREIGN KEY fk_payment_customer;
ALTER TABLE payment DROP FOREIGN KEY fk_payment_rental;
ALTER TABLE payment DROP FOREIGN KEY fk_payment_staff;
ALTER TABLE rental DROP FOREIGN KEY fk_rental_customer;
ALTER TABLE rental DROP FOREIGN KEY fk_rental_inventory;
ALTER TABL

You check the foreign key constraints are present in all the tables in the database

In [16]:
# OPTIONAL
sql_query='''SELECT 
  table_name,
  column_name,
  constraint_name,
  referenced_table_name,
  referenced_column_name
FROM
  information_schema.KEY_COLUMN_USAGE
WHERE
  REFERENCED_TABLE_SCHEMA = 'sakila'
  AND REFERENCED_TABLE_NAME IS NOT NULL;
'''


my_cursor.execute(sql_query)
foreign_keys=my_cursor.fetchall()
for row in foreign_keys:
    print(row)

{'table_name': 'address', 'column_name': 'city_id', 'constraint_name': 'fk_address_city', 'referenced_table_name': 'city', 'referenced_column_name': 'city_id'}
{'table_name': 'city', 'column_name': 'country_id', 'constraint_name': 'fk_city_country', 'referenced_table_name': 'country', 'referenced_column_name': 'country_id'}
{'table_name': 'customer', 'column_name': 'address_id', 'constraint_name': 'fk_customer_address', 'referenced_table_name': 'address', 'referenced_column_name': 'address_id'}
{'table_name': 'customer', 'column_name': 'store_id', 'constraint_name': 'fk_customer_store', 'referenced_table_name': 'store', 'referenced_column_name': 'store_id'}
{'table_name': 'film', 'column_name': 'language_id', 'constraint_name': 'fk_film_language', 'referenced_table_name': 'language', 'referenced_column_name': 'language_id'}
{'table_name': 'film', 'column_name': 'original_language_id', 'constraint_name': 'fk_film_language_original', 'referenced_table_name': 'language', 'referenced_colum

In [17]:
sql_query='''SELECT CONCAT('INSERT INTO actor(actor_id, first_name, last_name) VALUES(', actor_id, ', "', first_name, '", "', last_name, '");') 
FROM actor 
WHERE actor_id = 10;
'''

my_cursor.execute(sql_query)
cnt=my_cursor.fetchall()
for row in cnt:
    print(list(row.values())[0])

INSERT INTO actor(actor_id, first_name, last_name) VALUES(10, "CHRISTIAN", "GABLE");


5.7 Describing the Data Dictionary Views 

In [18]:
##Query to list data dictionary views and their purposes:
sql_query='''SELECT table_name, table_comment 
FROM information_schema.TABLES 
WHERE TABLE_SCHEMA = 'sakila';

'''


my_cursor.execute(sql_query)
views=my_cursor.fetchall()
for row in views:
    print(row)

{'table_name': 'actor', 'table_comment': ''}
{'table_name': 'actor_info', 'table_comment': 'VIEW'}
{'table_name': 'address', 'table_comment': ''}
{'table_name': 'category', 'table_comment': ''}
{'table_name': 'char_data_types', 'table_comment': ''}
{'table_name': 'city', 'table_comment': ''}
{'table_name': 'country', 'table_comment': ''}
{'table_name': 'customer', 'table_comment': ''}
{'table_name': 'customer_list', 'table_comment': 'VIEW'}
{'table_name': 'date_time_types', 'table_comment': ''}
{'table_name': 'film', 'table_comment': ''}
{'table_name': 'film_actor', 'table_comment': ''}
{'table_name': 'film_category', 'table_comment': ''}
{'table_name': 'film_list', 'table_comment': 'VIEW'}
{'table_name': 'film_text', 'table_comment': ''}
{'table_name': 'inventory', 'table_comment': ''}
{'table_name': 'language', 'table_comment': ''}
{'table_name': 'nicer_but_slower_film_list', 'table_comment': 'VIEW'}
{'table_name': 'number_data_types', 'table_comment': ''}
{'table_name': 'payment', '

In [19]:
##Query to describe the columns in a given data dictionary view:
sql_query='''SELECT COLUMN_NAME, COLUMN_COMMENT 
FROM information_schema.COLUMNS 
WHERE TABLE_SCHEMA = 'sakila' AND TABLE_NAME = 'actor';


'''


my_cursor.execute(sql_query)
views=my_cursor.fetchall()
for row in views:
    print(row)

{'COLUMN_NAME': 'actor_id', 'COLUMN_COMMENT': ''}
{'COLUMN_NAME': 'first_name', 'COLUMN_COMMENT': ''}
{'COLUMN_NAME': 'last_name', 'COLUMN_COMMENT': ''}
{'COLUMN_NAME': 'last_update', 'COLUMN_COMMENT': ''}


##### Chapter-6
##### Working with Strings

##### Counting the occurances of a Charecter in a string

- Counting the number of open brackets in a string.

Make sure to use a different connection and different cursor object

In [30]:
schekka_config = {
  'user': 'root',
  'password': 'root',
  'host': '127.0.0.1',
  'port': 8889,
  'database': 'Saiabhinav',
  'raise_on_warnings': True
}

In [31]:
schekka_db = mysql.connector.connect(**schekka_config)
# cursor object
new_cursor = schekka_db.cursor(dictionary=True)

In [32]:
new_cursor.execute('''SELECT * FROM emp''')
for emp_table in new_cursor:
    print(emp_table)


{'EMPNO': 7369, 'ENAME': 'SMITH', 'JOB': 'CLERK', 'MGR': '7902', 'HIREDATE': '17-Dec-05', 'SAL': 800, 'COMM': '', 'DEPTNO': 20}
{'EMPNO': 7499, 'ENAME': 'ALLEN', 'JOB': 'SALESMAN', 'MGR': '7698', 'HIREDATE': '20-Feb-06', 'SAL': 1600, 'COMM': '300', 'DEPTNO': 30}
{'EMPNO': 7521, 'ENAME': 'WARD', 'JOB': 'SALESMAN', 'MGR': '7698', 'HIREDATE': '22-Feb-06', 'SAL': 1250, 'COMM': '500', 'DEPTNO': 30}
{'EMPNO': 7566, 'ENAME': 'JONES', 'JOB': 'MANAGER', 'MGR': '7839', 'HIREDATE': '2-Apr-06', 'SAL': 2975, 'COMM': '', 'DEPTNO': 20}
{'EMPNO': 7654, 'ENAME': 'MARTIN', 'JOB': 'SALESMAN', 'MGR': '7698', 'HIREDATE': '28-Sep-06', 'SAL': 1250, 'COMM': '1400', 'DEPTNO': 30}
{'EMPNO': 7698, 'ENAME': 'BLAKE', 'JOB': 'MANAGER', 'MGR': '7839', 'HIREDATE': '1-May-06', 'SAL': 2850, 'COMM': '', 'DEPTNO': 30}
{'EMPNO': 7782, 'ENAME': 'CLARK', 'JOB': 'MANAGER', 'MGR': '7839', 'HIREDATE': '9-Jun-06', 'SAL': 2450, 'COMM': '', 'DEPTNO': 10}
{'EMPNO': 7788, 'ENAME': 'SCOTT', 'JOB': 'ANALYST', 'MGR': '7566', 'HIREDATE

In [33]:
new_cursor.execute('''SELECT * FROM dept''')
for dept_table in new_cursor:
    print(dept_table)


{'DEPTNO': 10, 'DNAME': 'ACCOUNTING', 'LOC': 'NEW YORK'}
{'DEPTNO': 20, 'DNAME': 'RESEARCH', 'LOC': 'DALLAS'}
{'DEPTNO': 30, 'DNAME': 'SALES', 'LOC': 'CHICAGO'}
{'DEPTNO': 40, 'DNAME': 'OPERATIONS', 'LOC': 'BOSTON'}


##### 6.4 Removing unwanted characters from string

- For Example, removing char 'P' from the name

In [34]:
new_cursor.execute('''select ename,
            replace(
            replace(
            replace(
            replace(
            replace(ename,'A',''),'E',''),'I',''),'O',''),'U','')
            as stripped1
            from emp;''')
for characters in new_cursor:
    print(characters)

{'ename': 'SMITH', 'stripped1': 'SMTH'}
{'ename': 'ALLEN', 'stripped1': 'LLN'}
{'ename': 'WARD', 'stripped1': 'WRD'}
{'ename': 'JONES', 'stripped1': 'JNS'}
{'ename': 'MARTIN', 'stripped1': 'MRTN'}
{'ename': 'BLAKE', 'stripped1': 'BLK'}
{'ename': 'CLARK', 'stripped1': 'CLRK'}
{'ename': 'SCOTT', 'stripped1': 'SCTT'}
{'ename': 'KING', 'stripped1': 'KNG'}
{'ename': 'TURNER', 'stripped1': 'TRNR'}
{'ename': 'ADAMS', 'stripped1': 'DMS'}
{'ename': 'JAMES', 'stripped1': 'JMS'}
{'ename': 'FORD', 'stripped1': 'FRD'}
{'ename': 'MILLER', 'stripped1': 'MLLR'}


Using the my_cursor object as we are connecting to sakila database to execute this query

In [35]:
my_cursor.execute(''' SELECT first_name, REPLACE(first_name, 'P', '') as strp1 FROM actor LIMIT 1''')
for repc in my_cursor:
    print(repc)

{'first_name': 'PENELOPE', 'strp1': 'ENELOE'}


6.5 Separating Numeric and Character Data - Skip as it uses Translate function

6.6 Determining Whether a String Is Alphanumeric

In [36]:
new_cursor.execute('''create view V as
    select ename as data
    from emp
    where deptno=10
    union all
    select concat(ename,', $',sal,'.00') as data
    from emp
    where deptno=20
    union all
    select concat(ename,deptno) as data
    from emp
    where deptno=30;''')
for V_view in new_cursor:
    print(V_view)

In [37]:
new_cursor.execute('''select * from V''')
for V_view in new_cursor:
    print(V_view)

{'data': 'CLARK'}
{'data': 'KING'}
{'data': 'MILLER'}
{'data': 'SMITH, $800.00'}
{'data': 'JONES, $2975.00'}
{'data': 'SCOTT, $3000.00'}
{'data': 'ADAMS, $1100.00'}
{'data': 'FORD, $3000.00'}
{'data': 'ALLEN30'}
{'data': 'WARD30'}
{'data': 'MARTIN30'}
{'data': 'BLAKE30'}
{'data': 'TURNER30'}
{'data': 'JAMES30'}


In [38]:
new_cursor.execute('''select data from V where data regexp '[^0-9a-zA-Z]' = 0;''')
for V_view in new_cursor:
    print(V_view)

{'data': 'CLARK'}
{'data': 'KING'}
{'data': 'MILLER'}
{'data': 'ALLEN30'}
{'data': 'WARD30'}
{'data': 'MARTIN30'}
{'data': 'BLAKE30'}
{'data': 'TURNER30'}
{'data': 'JAMES30'}


6.7 Extracting Initials from a Name

Using the my_cursor object as we are connecting to sakila database for this query

In [39]:
my_cursor.execute('''SELECT ((LENGTH('334 Munger (Monghyr) Lane')) - LENGTH(REPLACE('334 Munger (Monghyr) Lane', '(', ''))) / LENGTH("(") AS cnt ''')
for cnt in my_cursor:
    print(cnt)

{'cnt': Decimal('1.0000')}


In [40]:
my_cursor.execute('''
    SELECT * from actor
''')
for cnt in my_cursor:
    print(cnt)

{'actor_id': 1, 'first_name': 'PENELOPE', 'last_name': 'GUINESS', 'last_update': datetime.datetime(2006, 2, 15, 4, 34, 33)}
{'actor_id': 2, 'first_name': 'NICK', 'last_name': 'WAHLBERG', 'last_update': datetime.datetime(2006, 2, 15, 4, 34, 33)}
{'actor_id': 3, 'first_name': 'ED', 'last_name': 'CHASE', 'last_update': datetime.datetime(2006, 2, 15, 4, 34, 33)}
{'actor_id': 4, 'first_name': 'JENNIFER', 'last_name': 'DAVIS', 'last_update': datetime.datetime(2006, 2, 15, 4, 34, 33)}
{'actor_id': 5, 'first_name': 'JOHNNY', 'last_name': 'LOLLOBRIGIDA', 'last_update': datetime.datetime(2006, 2, 15, 4, 34, 33)}
{'actor_id': 6, 'first_name': 'BETTE', 'last_name': 'NICHOLSON', 'last_update': datetime.datetime(2006, 2, 15, 4, 34, 33)}
{'actor_id': 7, 'first_name': 'GRACE', 'last_name': 'MOSTEL', 'last_update': datetime.datetime(2006, 2, 15, 4, 34, 33)}
{'actor_id': 8, 'first_name': 'MATTHEW', 'last_name': 'JOHANSSON', 'last_update': datetime.datetime(2006, 2, 15, 4, 34, 33)}
{'actor_id': 9, 'first

In [41]:
my_cursor.execute('''
    SELECT first_name, last_name, CONCAT(SUBSTRING(SUBSTRING_INDEX(first_name, ' ', 1), 1, 1), '.', SUBSTRING(SUBSTRING_INDEX(last_name, ' ', -1), 1, 1)) AS initials
    FROM actor;
''')
for cnt in my_cursor:
    print(cnt)

{'first_name': 'PENELOPE', 'last_name': 'GUINESS', 'initials': 'P.G'}
{'first_name': 'NICK', 'last_name': 'WAHLBERG', 'initials': 'N.W'}
{'first_name': 'ED', 'last_name': 'CHASE', 'initials': 'E.C'}
{'first_name': 'JENNIFER', 'last_name': 'DAVIS', 'initials': 'J.D'}
{'first_name': 'JOHNNY', 'last_name': 'LOLLOBRIGIDA', 'initials': 'J.L'}
{'first_name': 'BETTE', 'last_name': 'NICHOLSON', 'initials': 'B.N'}
{'first_name': 'GRACE', 'last_name': 'MOSTEL', 'initials': 'G.M'}
{'first_name': 'MATTHEW', 'last_name': 'JOHANSSON', 'initials': 'M.J'}
{'first_name': 'JOE', 'last_name': 'SWANK', 'initials': 'J.S'}
{'first_name': 'CHRISTIAN', 'last_name': 'GABLE', 'initials': 'C.G'}
{'first_name': 'ZERO', 'last_name': 'CAGE', 'initials': 'Z.C'}
{'first_name': 'KARL', 'last_name': 'BERRY', 'initials': 'K.B'}
{'first_name': 'UMA', 'last_name': 'WOOD', 'initials': 'U.W'}
{'first_name': 'VIVIEN', 'last_name': 'BERGEN', 'initials': 'V.B'}
{'first_name': 'CUBA', 'last_name': 'OLIVIER', 'initials': 'C.O'}


6.8 Ordering by Parts of a String

In [42]:
## ordering by last two characters
new_cursor.execute('''select ename from emp order by substr(ename,length(ename)-1,2);''')
for sub in new_cursor:
    print(sub)

{'ename': 'ALLEN'}
{'ename': 'TURNER'}
{'ename': 'MILLER'}
{'ename': 'JONES'}
{'ename': 'JAMES'}
{'ename': 'MARTIN'}
{'ename': 'BLAKE'}
{'ename': 'ADAMS'}
{'ename': 'KING'}
{'ename': 'WARD'}
{'ename': 'FORD'}
{'ename': 'CLARK'}
{'ename': 'SMITH'}
{'ename': 'SCOTT'}


In [43]:
my_cursor.execute(''' SELECT title FROM film ORDER BY SUBSTR(title, LENGTH(title) - 1) LIMIT 5; ''')
conc_res = my_cursor.fetchall()
print(conc_res)

[{'title': 'FROST HEAD'}, {'title': 'CATCH AMISTAD'}, {'title': 'PACIFIC AMISTAD'}, {'title': 'COMA HEAD'}, {'title': 'ROUGE SQUAD'}]


6.9 Ordering by a Number in a String - Cannot execute as MySQL does not provide the TRANSLATE function

6.10 Creating a Delimited List from Table Rows

In [44]:
new_cursor.execute('''select deptno, group_concat(ename order by empno separator ',') as emps from emp group by deptno;''')
for delimit in new_cursor:
    print(delimit)

{'deptno': 10, 'emps': 'CLARK,KING,MILLER'}
{'deptno': 20, 'emps': 'SMITH,JONES,SCOTT,ADAMS,FORD'}
{'deptno': 30, 'emps': 'ALLEN,WARD,MARTIN,BLAKE,TURNER,JAMES'}


6.11 Converting Delimited Data into a Multivalued IN-List

In [45]:
## here, the IN LIST ( '7654,7698,7782,7788' ) is considered to be a string rather than a comma delimited list
new_cursor.execute('''select empno, ename,sal,deptno
    from emp
    where empno in ( 7654,7698,7782,7788 );''')
for delimit in new_cursor:
    print(delimit)

{'empno': 7654, 'ename': 'MARTIN', 'sal': 1250, 'deptno': 30}
{'empno': 7698, 'ename': 'BLAKE', 'sal': 2850, 'deptno': 30}
{'empno': 7782, 'ename': 'CLARK', 'sal': 2450, 'deptno': 10}
{'empno': 7788, 'ename': 'SCOTT', 'sal': 3000, 'deptno': 20}


In [46]:
new_cursor.execute('''select ename,sal,deptno
    from emp
    where empno= 7654;''')
for delimit in new_cursor:
    print(delimit)

{'ename': 'MARTIN', 'sal': 1250, 'deptno': 30}


In [47]:
new_cursor.execute('''CREATE TEMPORARY TABLE temp_numbers (id INT);''')
for temp in new_cursor:
    print(temp)

In [48]:
new_cursor.execute('''INSERT INTO temp_numbers (id) VALUES 
(1), (2), (3), (4), (5), (6), (7), (8), (9), (10),
(11), (12), (13), (14), (15), (16), (17), (18), (19), (20);''')
for temp in new_cursor:
    print(temp)

In [49]:
new_cursor.execute('''select empno, ename, sal, deptno 
    from emp
    where empno in
    ( select substring_index(
     substring_index(list.vals,',',iter.id),',',-1) empno
    from temp_numbers as iter,
    (select '7654,7698,7782,7788' as vals
    from (select 1) as dummy_table
    )list
    where iter.id <=
    (length(list.vals)-length(replace(list.vals,',','')))+1
    );''')
for delimit in new_cursor:
    print(delimit)

{'empno': 7654, 'ename': 'MARTIN', 'sal': 1250, 'deptno': 30}
{'empno': 7698, 'ename': 'BLAKE', 'sal': 2850, 'deptno': 30}
{'empno': 7782, 'ename': 'CLARK', 'sal': 2450, 'deptno': 10}
{'empno': 7788, 'ename': 'SCOTT', 'sal': 3000, 'deptno': 20}


6.12 Alphabetizing a String

In [50]:
new_cursor.execute('''select ename, group_concat(c order by c separator '') AS sorted_ename
    from (
    select ename, substr(a.ename,iter.id,1) c
    from emp a, temp_numbers iter
    WHERE iter.id <= LENGTH(a.ename)
    ) x
    GROUP BY ename;''')
for alpha in new_cursor:
    print(alpha)

{'ename': 'ADAMS', 'sorted_ename': 'AADMS'}
{'ename': 'ALLEN', 'sorted_ename': 'AELLN'}
{'ename': 'BLAKE', 'sorted_ename': 'ABEKL'}
{'ename': 'CLARK', 'sorted_ename': 'ACKLR'}
{'ename': 'FORD', 'sorted_ename': 'DFOR'}
{'ename': 'JAMES', 'sorted_ename': 'AEJMS'}
{'ename': 'JONES', 'sorted_ename': 'EJNOS'}
{'ename': 'KING', 'sorted_ename': 'GIKN'}
{'ename': 'MARTIN', 'sorted_ename': 'AIMNRT'}
{'ename': 'MILLER', 'sorted_ename': 'EILLMR'}
{'ename': 'SCOTT', 'sorted_ename': 'COSTT'}
{'ename': 'SMITH', 'sorted_ename': 'HIMST'}
{'ename': 'TURNER', 'sorted_ename': 'ENRRTU'}
{'ename': 'WARD', 'sorted_ename': 'ADRW'}


6.13 Identifying Strings That Can Be Treated as Numbers


In [51]:
new_cursor.execute('''create view P as
    select concat(
    substr(ename,1,2),
    replace(cast(deptno as char(4)),' ',''),
    substr(ename,3,2)
    ) as mixed
    from emp
    where deptno = 10
    union all
    select replace(cast(empno as char(4)), ' ', '')
    from emp where deptno = 20
    union all
    select ename from emp where deptno = 30;''')
for View_P in new_cursor:
    print(View_P)

In [52]:
new_cursor.execute('''select * from P''')
for View_P in new_cursor:
    print(View_P)

{'mixed': 'CL10AR'}
{'mixed': 'KI10NG'}
{'mixed': 'MI10LL'}
{'mixed': '7369'}
{'mixed': '7566'}
{'mixed': '7788'}
{'mixed': '7876'}
{'mixed': '7902'}
{'mixed': 'ALLEN'}
{'mixed': 'WARD'}
{'mixed': 'MARTIN'}
{'mixed': 'BLAKE'}
{'mixed': 'TURNER'}
{'mixed': 'JAMES'}


In [53]:
new_cursor.execute('''select cast(group_concat(c order by pos separator '') as unsigned)
        as MIXED1
        from (
        select P.mixed, iter.id AS pos, substr(P.mixed,iter.id,1) as c from P,
        temp_numbers as iter where iter.id <= length(P.mixed)
        and ascii(substr(P.mixed,iter.id,1)) between 48 and 57)y
        
        group by mixed
        order by 1;''')
for str in new_cursor:
    print(str)

{'MIXED1': 10}
{'MIXED1': 10}
{'MIXED1': 10}
{'MIXED1': 7369}
{'MIXED1': 7566}
{'MIXED1': 7788}
{'MIXED1': 7876}
{'MIXED1': 7902}


6.14 Extracting the nth Delimited Substring

In [54]:
new_cursor.execute('''CREATE TABLE permanent_numbers (id INT PRIMARY KEY);''')
for str in new_cursor:
    print(str)

In [55]:
new_cursor.execute('''INSERT INTO permanent_numbers VALUES (1), (2), (3),(4), (5), (6), (7), (8), (9), (10),
(11), (12), (13), (14), (15), (16), (17), (18), (19), (20);
''')
for temp in new_cursor:
    print(temp)

In [56]:
new_cursor.execute('''CREATE VIEW C AS
    SELECT 'mo,larry,curly' AS name
    FROM permanent_numbers
    UNION ALL
    SELECT 'tina,gina,jaunita,regina,leena' AS name
    FROM permanent_numbers;''')
for View_C in new_cursor:
    print(View_C)

In [57]:
new_cursor.execute('''select distinct  * from C;''')
for View_C in new_cursor:
    print(View_C)

{'name': 'mo,larry,curly'}
{'name': 'tina,gina,jaunita,regina,leena'}


In [58]:
new_cursor.execute('''Select distinct name
        from (
        select iter.id AS pos,
        substring_index(
        substring_index(src.name,',',iter.id),',',-1) AS name
        from C src,
        temp_numbers iter
        where iter.id <=
        length(src.name)-length(replace(src.name,',',''))
        ) x
        where pos = 2 LIMIT 2;''')
for View_C in new_cursor:
    print(View_C)

{'name': 'larry'}
{'name': 'gina'}


In [59]:
new_cursor.execute('''select iter.id AS pos, src.name
from temp_numbers iter, C src
where iter.id <= length(src.name) - length(replace(src.name, ',', '')) LIMIT 1;''')
for View_C in new_cursor:
    print(View_C)

{'pos': 1, 'name': 'mo,larry,curly'}


In [60]:
new_cursor.execute('''select iter.id AS pos, src.name AS name1,
    substring_index(src.name,',',iter.id) AS name2,
    substring_index(
    substring_index(src.name,',',iter.id),',',-1) AS name3
    from temp_numbers iter,
    C src
    where iter.id <= length(src.name) - length(replace(src.name,',','')) LIMIT 2;''')
for View_C in new_cursor:
    print(View_C)

{'pos': 1, 'name1': 'mo,larry,curly', 'name2': 'mo', 'name3': 'mo'}
{'pos': 2, 'name1': 'mo,larry,curly', 'name2': 'mo,larry', 'name3': 'larry'}


6.15 Parsing an IP Address

In [61]:
new_cursor.execute('''select substring_index(substring_index(y.ip,'.',1),'.',-1) a,
       substring_index(substring_index(y.ip,'.',2),'.',-1) b,
       substring_index(substring_index(y.ip,'.',3),'.',-1) c,
       substring_index(substring_index(y.ip,'.',4),'.',-1) d
from (select '92.111.0.2' as ip from temp_numbers LIMIT 1) y;
''')
for address in new_cursor:
    print(address)

{'a': '92', 'b': '111', 'c': '0', 'd': '2'}


6.16 Comparing Strings by Sound

In [62]:
new_cursor.execute('''CREATE TABLE author_names (
    a_name VARCHAR(50));''')
for author in new_cursor:
    print(author)

In [63]:
new_cursor.execute('''INSERT INTO author_names (a_name) VALUES
('Johnson'),
('Jonson'),
('Jonsen'),
('Jensen'),
('Johnsen'),
('Shakespeare'),
('Shakspear'),
('Shaekspir'),
('Shakespar');
''')

for author in new_cursor:
    print(author)

In [64]:
new_cursor.execute('''select distinct an1.a_name as name1, an2.a_name as name2,
    SOUNDEX(an1.a_name) as Soundex_Name
    from author_names an1
    join author_names an2
    on (SOUNDEX(an1.a_name)=SOUNDEX(an2.a_name)
    and an1.a_name!=  an2.a_name);''')

for author in new_cursor:
    print(author)

{'name1': 'Jonson', 'name2': 'Johnson', 'Soundex_Name': 'J525'}
{'name1': 'Jonsen', 'name2': 'Johnson', 'Soundex_Name': 'J525'}
{'name1': 'Jensen', 'name2': 'Johnson', 'Soundex_Name': 'J525'}
{'name1': 'Johnsen', 'name2': 'Johnson', 'Soundex_Name': 'J525'}
{'name1': 'Johnson', 'name2': 'Jonson', 'Soundex_Name': 'J525'}
{'name1': 'Jonsen', 'name2': 'Jonson', 'Soundex_Name': 'J525'}
{'name1': 'Jensen', 'name2': 'Jonson', 'Soundex_Name': 'J525'}
{'name1': 'Johnsen', 'name2': 'Jonson', 'Soundex_Name': 'J525'}
{'name1': 'Johnson', 'name2': 'Jonsen', 'Soundex_Name': 'J525'}
{'name1': 'Jonson', 'name2': 'Jonsen', 'Soundex_Name': 'J525'}
{'name1': 'Jensen', 'name2': 'Jonsen', 'Soundex_Name': 'J525'}
{'name1': 'Johnsen', 'name2': 'Jonsen', 'Soundex_Name': 'J525'}
{'name1': 'Johnson', 'name2': 'Jensen', 'Soundex_Name': 'J525'}
{'name1': 'Jonson', 'name2': 'Jensen', 'Soundex_Name': 'J525'}
{'name1': 'Jonsen', 'name2': 'Jensen', 'Soundex_Name': 'J525'}
{'name1': 'Johnsen', 'name2': 'Jensen', 'Soun

6.17 Finding Text Not Matching a Pattern:  Skip as this is a specific Oracle Database query.

##### Chapter-7
- 7.1 Computing an Average

In [65]:
# cursor object for sakila database
mydb = mysql.connector.connect(**config)
my_cursor = mydb.cursor(dictionary=True)

In [66]:
my_cursor.execute('''SELECT AVG(amount) AS avg_amount FROM payment''')
avg_res = my_cursor.fetchall()
print(avg_res)

[{'avg_amount': Decimal('4.201356')}]


In [67]:
my_cursor.execute('''SELECT AVG(amount) AS avg_amount FROM payment GROUP BY payment_date LIMIT 5''')
avg_res = my_cursor.fetchall()
print(avg_res)

[{'avg_amount': Decimal('2.990000')}, {'avg_amount': Decimal('2.990000')}, {'avg_amount': Decimal('3.990000')}, {'avg_amount': Decimal('4.990000')}, {'avg_amount': Decimal('6.990000')}]


In [68]:
my_cursor.execute(''' SELECT MAX(amount) AS MAX_PAYMENT FROM payment ''')
minmax_res = my_cursor.fetchall()
print(minmax_res)

[{'MAX_PAYMENT': Decimal('11.99')}]


In [69]:
my_cursor.execute(''' SELECT SUM(amount) AS TOTAL_PAYMENT FROM payment ''')
minmax_res = my_cursor.fetchall()
print(minmax_res)

[{'TOTAL_PAYMENT': Decimal('67406.56')}]


In [70]:
my_cursor.execute(''' SELECT COUNT(*) AS total FROM film ''')
total_rows = my_cursor.fetchall()
print(total_rows)

[{'total': 1000}]


In [71]:
# This will exclude all the Max and Min values (if the column contains more than one max and min)
my_cursor.execute(''' SELECT AVG(amount) AS avg_without_high_low_values FROM payment WHERE amount NOT IN ((SELECT MAX(amount) FROM payment), (SELECT MIN(amount) FROM payment)) ''')
avg_w = my_cursor.fetchall()
print(avg_w)

[{'avg_without_high_low_values': Decimal('4.202790')}]


In [72]:
# Use this to exclude only one set of Max and Min
my_cursor.execute(''' SELECT (SUM(amount) - MIN(amount) - MAX(amount)) / (COUNT(*) - 2) AS avg_wmm FROM payment''')
avg_without_max_min = my_cursor.fetchall()
print(avg_without_max_min)

[{'avg_wmm': Decimal('4.201133')}]


7.6 Generating a Running: Run the queries from 7.6 in sqlite as these queries use window functions which is not supported by MySQL Workbench

In [73]:
import sqlite3
import pandas as pd

In [74]:
conn = sqlite3.connect('week1.db')
c = conn.cursor()

In [75]:
read_emp = pd.read_csv(r'/Users/saiabhinavchekka/Desktop/emp.csv')
read_emp.to_sql('emp', conn, if_exists='replace', index = False) # Insert the values from the csv file into the table 'emp'

14

In [76]:
read_dept = pd.read_csv(r'/Users/saiabhinavchekka/Desktop/dept.csv')
read_dept.to_sql('dept', conn, if_exists='replace', index = False) # Insert the values from the csv file into the table 'dept'

4

7.6 Generating a Running Total

In [77]:
c.execute('''
select ename, sal,
sum(sal) over (order by sal,empno) as running_total
from emp
order by 2;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

,ENAME,SAL,running_total
0,SMITH,800,800
1,JAMES,950,1750
2,ADAMS,1100,2850
3,WARD,1250,4100
4,MARTIN,1250,5350
5,MILLER,1300,6650
6,TURNER,1500,8150
7,ALLEN,1600,9750
8,CLARK,2450,12200
9,BLAKE,2850,15050


In [78]:
c.execute('''
select empno, sal,
sum(sal)over(order by sal,empno) as running_total1,
sum(sal)over(order by sal) as running_total2
from emp
order by 2;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

,EMPNO,SAL,running_total1,running_total2
0,7369,800,800,800
1,7900,950,1750,1750
2,7876,1100,2850,2850
3,7521,1250,4100,5350
4,7654,1250,5350,5350
5,7934,1300,6650,6650
6,7844,1500,8150,8150
7,7499,1600,9750,9750
8,7782,2450,12200,12200
9,7698,2850,15050,15050


7.7 Generating a Running Product

In [79]:
#if you are facing issues in exp and ln function, it is because of the version of MySQL that you are using.
import math

In [80]:
conn.create_function("exp", 1, math.exp)
conn.create_function("ln", 1, math.log)
c= conn.cursor()

In [81]:
c.execute('''
select empno,ename,sal,
exp(sum(ln(sal))over(order by sal,empno)) as running_prod
from emp
where deptno = 10;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

,EMPNO,ENAME,SAL,running_prod
0,7934,MILLER,1300,1.300000e+03
1,7782,CLARK,2450,3.185000e+06
2,7839,KING,5000,1.592500e+10


7.8 Smoothing a Series of Values - it is okay if you skip

7.9 Calculating a Mode

In [82]:
c.execute('''
select sal
from emp
where deptno = 20
order by sal;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

,SAL
0,800
1,1100
2,2975
3,3000
4,3000


In [83]:
# Mode is 3000
c.execute('''
    select sal
    from (
    select sal,
     dense_rank()over( order by cnt desc) as rnk
    from (
    select sal, count(*) as cnt
    from emp
    where deptno = 20
    group by sal
    ) x
    ) y
    where rnk = 1;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

,sal
0,3000


In [84]:
c.execute('''
select sal,
dense_rank()over(order by cnt desc) as rnk
from (
select sal,count(*) as cnt
from emp
where deptno = 20
group by sal
) x;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

,sal,rnk
0,3000,1
1,800,2
2,1100,2
3,2975,2


7.10 Calculating a Median


In [85]:
c.execute('''
select sal
from emp
where deptno = 20
order by sal;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

,SAL
0,800
1,1100
2,2975
3,3000
4,3000


In [86]:
c.execute('''
with rank_tab (sal, rank_sal) as
(
select sal, cume_dist() over (order by sal)
from emp
where deptno=20
),
inter as
(
select sal, rank_sal from rank_tab
where rank_sal>=0.5
union
select sal, rank_sal from rank_tab
where rank_sal<=0.5
)
select avg(sal) as MedianSal
from inter;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

,MedianSal
0,1968.75


7.11 Determining the Percentage of a Total

In [87]:
c.execute('''
select *
from emp where deptno=10;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7782,CLARK,MANAGER,7839.0,9-Jun-06,2450,None,10
1,7839,KING,PRESIDENT,NaN,17-Nov-06,5000,None,10
2,7934,MILLER,CLERK,7782.0,23-Jan-07,1300,None,10


In [88]:
c.execute('''
select (CAST(sum(case when deptno = 10 then sal end) AS FLOAT)/CAST(sum(sal) AS FLOAT)
)*100 as pct
from emp;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

,pct
0,30.146425


In [89]:
c.execute('''
select sum(case when deptno = 10 then sal end) as d10,
sum(sal)
from emp;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

,d10,sum(sal)
0,8750,29025


In [90]:
c.execute('''
select (cast(
sum(case when deptno = 10 then sal end)
as FLOAT))/CAST(sum(sal) AS FLOAT
)*100 as pct
from emp;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

,pct
0,30.146425


In [91]:
c.execute('''
select deptno,
sum(sal)over() total,
sum(sal)over(partition by deptno) d10
from emp;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

,DEPTNO,total,d10
0,10,29025,8750
1,10,29025,8750
2,10,29025,8750
3,20,29025,10875
4,20,29025,10875
5,20,29025,10875
6,20,29025,10875
7,20,29025,10875
8,30,29025,9400
9,30,29025,9400


In [92]:
c.execute('''
select deptno,
sum(sal)over() total,
sum(sal)over(partition by deptno) d10
from emp
where deptno=10;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

,DEPTNO,total,d10
0,10,8750,8750
1,10,8750,8750
2,10,8750,8750


7.12 Aggregating Nullable Columns

In [93]:
c.execute('''
select avg(coalesce(comm,0)) as avg_comm
from emp
where deptno=30;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

,avg_comm
0,366.666667


In [94]:
c.execute('''
select avg(comm)
from emp
where deptno=30;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

,avg(comm)
0,550.0


In [95]:
c.execute('''
select ename, comm
from emp
where deptno=30
order by comm desc;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

,ENAME,COMM
0,MARTIN,1400.0
1,WARD,500.0
2,ALLEN,300.0
3,TURNER,0.0
4,BLAKE,NaN
5,JAMES,NaN


7.13 Computing Averages Without High and Low Values

In [96]:
c.execute('''
select avg(sal)
from emp
where sal not in (
(select min(sal) from emp),
(select max(sal) from emp)
);
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

,avg(sal)
0,1935.416667


In [97]:
c.execute('''
select (sum(sal)-min(sal)-max(sal))/(count(*)-2) AS cnt
from emp;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

,cnt
0,1935


In [98]:
c.execute('''
select sal, min(sal)over() min_sal, max(sal)over() max_sal
from emp;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

,SAL,min_sal,max_sal
0,800,800,5000
1,1600,800,5000
2,1250,800,5000
3,2975,800,5000
4,1250,800,5000
5,2850,800,5000
6,2450,800,5000
7,3000,800,5000
8,5000,800,5000
9,1500,800,5000


Skip 7.14, 7.15 as it uses traslate function

7.16 Finding Outliers Using the Median Absolute
Deviation

In [99]:
c.execute('''
    with rank_tab (sal, rank_sal) as (
    select sal, cume_dist() over (order by sal)
    from emp),
    inter as
    (
    select sal, rank_sal from rank_tab
    where rank_sal>=0.5
    union
    select sal, rank_sal from rank_tab
    where rank_sal<=0.5
    )
    ,
    medianSal (medianSal) as
    (
    select (max(sal)+min(sal))/2
    from inter),
    deviationSal (Sal,deviationSal) as
    (select Sal,abs(sal-medianSal)
    from emp join medianSal
    on 1=1
    )
    ,
    distDevSal (sal,deviationSal,distDeviationSal) as
    (
    select sal,deviationSal,cume_dist() over (order by deviationSal)
    from deviationSal
    ),
    DevInter (DevInter, sal) as
    (
    select min(deviationSal), sal
    from distDevSal
    where distDeviationSal >= 0.5
    union
    select max(DeviationSal), sal
    from distDevSal
    where distDeviationSal <= 0.5
    ),
    MAD (MedianAbsoluteDeviance) as
    (
    select abs(emp.sal-(min(devInter)+max(devInter))/2)
    from emp join DevInter on 1=1
    )
    select emp.sal,MedianAbsoluteDeviance,
    (emp.sal-deviationSal)/MedianAbsoluteDeviance as ratio
    from (emp join MAD on 1=1)
    join deviationSal on emp.sal=deviationSal.sal;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

,SAL,MedianAbsoluteDeviance,ratio
0,800,600,-2
1,1600,600,0
2,1250,600,0
3,1250,600,0
4,2975,600,4
5,1250,600,0
6,1250,600,0
7,2850,600,4
8,2450,600,3
9,3000,600,4


Skip 7.17 Finding Anomalies Using Benford’s Law

In [100]:
conn.close()